# SI 2020/1 - Sistemas a Decisão - Exame 1

Como parte da avaliação da disciplina de Sistema a Decisão ministrada pelo Prof.: Jean Caminha, foi proposta a seguinte atividade:

## EXAME - Implementação de um SAD
Implemente um SAD aplicando os conceitos das técnicas de desenvolvimento de sistemas de apoio a decisão em uma base de dados pública.

Na implentação deve estar clara os seguintes pontos:

- Qual decisão este sistema vai apoiar;

- O processo de ETL;

- Qual o modelo utilizado e por que?

- Visualização dos dados

- Conclusão e/ou melhor decisão.

Observaçoes:

1. Você pode utilizar qualquer linguagem de implantação e BD que você domine. Sugiro no entanto utilizar a plataforma Kaggle ou Google Colab e submeta o link como resposta (maneira mais fácil).

2. São fontes de dados:

- http://dados.gov.br/ 

- https://github.com/awesomedata/awesome-public-datasets

3. Um trabalho por aluno(a). Trabalhos iguais terão a nota zero.

4. Utilize o forum para dúvias sobre esta tarefa.

Bom trabalho à todos.

Jean

Explorando pelos bancos dados disponíveis recomendados pelo professor e, devido à minha experiência com a linguagem de programação [C#](https://docs.microsoft.com/pt-br/dotnet/csharp/), procurei nas documentações da [Microsof](https://docs.microsoft.com/en-us/documentation/) se havia alguma maneira de praticar técnicas de aprendizado de máquina e, felizmente, encontrei o framework [ML.NET](https://docs.microsoft.com/en-us/dotnet/machine-learning/) que permite criar modelos customizados de aprendizado de máquina para ser consumido pelas aplicações [.NET](https://docs.microsoft.com/en-us/dotnet/).

Lendo a documentação também tive o conhecimento da utilização da ferramenta [ML.NET Model Builder](https://docs.microsoft.com/en-us/dotnet/machine-learning/automate-training-with-model-builder) que basicamente é uma extensão gráfica para IDE da Microsoft, [Visual Studio](https://visualstudio.microsoft.com/), que permite a criação automática desses modelos preenchendo alguns requisitos:

- Cenário: Basicamente é em qual tipo de aprendizado que o seu modelo deverá se basear. Dentre as opções havia classificação de imagens usando o [Azure](https://azure.microsoft.com/pt-br/) como unidade de processamento, predição de preços usando algoritmos de regressão, recomendação de filmes e também classificação de sentimentos em textos que foi o cenário selecionado por mim.

- Dados: O [ML.NET Model Builder](https://docs.microsoft.com/en-us/dotnet/machine-learning/automate-training-with-model-builder) tem suporte a uma vasta gama de arquivos suportados dentre eles podemos citar CSV, TSC e outros além do suporte a banco de dados do SQL Server.

- Treinamento: De longe o processo mais demorado que consiste em selecionar quais colunas, ou *inouts* melhor dizendo, que serão utilizados durante o treinamento, bem como qual valor de predição que desejo alcançar. O treinamento támbem permite escolher o tempo máximo do processamento que consiste na execução e comparação dos algoritmos de aprendizado de máquina classificando-os e selecionando aquele que obteve o melhor desempenho e eficácia nos resultados.

- Avaliação: Esse processo se limita apenas em testar se o modelo criado está funcionando adequadamente preenchendo lacunas necessárias para que a modelo posso realizar a predição.

Nos momentos iniciais das minhas tentativas de construção, em menos 30 minutos já tinha modelo pronto para ser usado, porém como pré-requisito da atividade necessitava a demonstração dos valores e suas devidas tranformações, tive que recorrer a procurar um modelo que se assemelha-se ao [Jupyter](https://jupyter.org/) para realizar o *storytelling* dos meus dados. Fazendo pesquisas tive a descoberta que o Jupyter era um framework versátil que permitia sua utilização com outras linguagens além do [Python](https://www.python.org/) e além disso, devido ao [.NET Interactive](https://devblogs.microsoft.com/dotnet/net-interactive-is-here-net-notebooks-preview-2/), a linguagem [C#](https://docs.microsoft.com/pt-br/dotnet/csharp/) poderia ser utilizada para a criação de *notebooks* dentro do Jupyter que será o modelo utilizado nesse exame.

## Qual decisão este sistema vai apoiar

Como dito anteriormente, o modelo irá se basear no processamento e classificação de sentimentos textos (em inglês) utilizando os dados disponíveis em [Sentiment140](http://help.sentiment140.com/for-students/).

## O processo de ETL

Como os dados providos pelo [Sentiment140](http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip) que se resumem em uma arquivo CSV com mais de 
1.5 milhão de linhas sem cabeçalho, um *script* é executado adicionando cabeçalhos customizados para a representação facilitada das colunas. Entretanto, na descrição do *dataset* detalha que os dados foram analisados e distribuídos com os seus devidos valores de sentimento sendo 0 para negativo, 2 para neutro e 4 para positivo. Porém, ao realizar estudos estatísticos nos dados fornecidos, pode-se concluir que os dados estavam polarizados de forma binária com textos e sentimentos atribuídos em valores variando entre apenas 0 e 2 (ver abaixo). Logo, o modelo de ETL ficou assim:

Para a realização da atividade e a execução dos scripts, foram utilizados os seguintes pacotes externos providos pelo [Nuget](https://www.nuget.org/):

- [Microsoft.ML](https://www.nuget.org/packages/Microsoft.ML/): Fornece classes e métodos para a criação, treinamento e avaliação de modelos de aprendizado de máquina.

- [Microsoft.Data.Analysis](https://www.nuget.org/packages/Microsoft.Data.Analysis/): Fornece classes e métodos para a manipulação de dados de forma semelhante ao presente no framework [pandas](https://pandas.pydata.org/), mas o pacote ainda está em desenvolvimento. Utilizado como uma alternativa menos verbosa para a auxiliar na plotagem dos dados.

- [XPlot.Plotly](https://www.nuget.org/packages/XPlot.Plotly): Possui componentes auxiliares para a plotagem de dados em forma de gráficos além de possuir implementações similares das ferramentas do [Google Charts](https://developers.google.com/chart/) com suporte às aplicações [.NET](https://docs.microsoft.com/en-us/dotnet/).

In [1]:
#r "nuget:Microsoft.ML, 1.5.2"
#r "nuget:Microsoft.Data.Analysis, 0.4.0"

using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.Data;
using System.IO;
using System.IO.Compression;
using System.Net;
using XPlot.Plotly;

In [1]:
public class ModelInput
{
    [ColumnName("Sentiment"), LoadColumn(0)]
    public string Sentiment { get; set; }


    [ColumnName("ID"), LoadColumn(1)]
    public float ID { get; set; }


    [ColumnName("Date"), LoadColumn(2)]
    public string Date { get; set; }


    [ColumnName("Query"), LoadColumn(3)]
    public string Query { get; set; }


    [ColumnName("User"), LoadColumn(4)]
    public string User { get; set; }


    [ColumnName("Text"), LoadColumn(5)]
    public string Text { get; set; }
}


E para o modelo de resultado, ele resume basicamente em uma **string** representando o nome e um *array* do tipo **float** do valor que o modelo irá tentar realizar a predição. Logo, ficando assim:

In [1]:
public class ModelOutput
{
    [ColumnName("PredictedLabel")]
    public String Prediction { get; set; }
    public float[] Score { get; set; }
}

Para a execução do script, foram definidos algumas constantes:

- DATASET_ABSOLUTEURI: URL dos dados da [Sentiment140](http://help.sentiment140.com/for-students/)
- MODEL_FILEPATH: Nome do arquivo do modelo de aprendizado de máquina já treinado gerado pelo script.
- TRAIN_DATA_FILEPATH: Nome do arquivo CSV utilizado para o treinamento do modelo, presente dentro do ZIP fornecido pela [Sentiment140](http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip)
- TEST_DATA_FILEPATH: Nome do arquvido CSV fornecido pela [Sentiment140](http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip) para a realização de teste. Não utilizado no script.

In [1]:
public const string DATASET_ABSOLUTEURI = "http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip";
public const string MODEL_FILEPATH = "MLModel.zip";
public const string TRAIN_DATA_FILEPATH = "training.1600000.processed.noemoticon.csv";
public const string TEST_DATA_FILEPATH = "testdata.manual.2009.06.14.csv";

Define métodos para o download(se necessário) do *dataset* bem como também a inserção do cabeçalho faltante no arquivo CSV fornecido.

In [1]:
public void AddCsvHeaders(){
    var headers = new[]{
        "\"Sentiment\"",
        "\"ID\"",
        "\"Date\"",
        "\"Query\"",
        "\"User\"",
        "\"Text\"",
    };

    var headerLine = string.Join(',', headers);

    var lines = File.ReadAllLines(Path.GetFileName(TRAIN_DATA_FILEPATH)).ToList();
    lines.Insert(0, headerLine);

    File.WriteAllLines(TRAIN_DATA_FILEPATH, lines);
}

public void LoadDataSet(){
    if (!File.Exists(Path.GetFileName(DATASET_ABSOLUTEURI))){
        using (var webClient = new WebClient()){
            webClient.DownloadFile(DATASET_ABSOLUTEURI, Path.GetFileName(DATASET_ABSOLUTEURI));
        }
    }

    if (!File.Exists(TRAIN_DATA_FILEPATH)){
        ZipFile.ExtractToDirectory(Path.GetFileName(DATASET_ABSOLUTEURI), Environment.CurrentDirectory);
        AddCsvHeaders();
    }
}

LoadDataSet();

Define um método que registra um formator dos dados para serem listados devidamente com suas colunas e seus respectivos dados que por padrão retorna 20 tupplas.

In [1]:
public void AddDataFrameFormatter(int count = 20){
    Formatter.Register<DataFrame>((df, writer) =>
    {
        var headers = new List<IHtmlContent>();
        headers.Add(th(i("Index")));
        headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    
        var rows = new List<List<IHtmlContent>>();
        for (var i = 0; i < Math.Min(count, df.Rows.Count); i++)
        {
            var cells = new List<IHtmlContent>();
    
            cells.Add(td(i));
    
            foreach (var obj in df.Rows[i])
            {
                cells.Add(td(obj));
            }
            
            rows.Add(cells);
        }
    
        var t = table(
            thead(
                headers),
            tbody(
                rows.Select(
                    r => tr(r))));
    
        writer.Write(t);
    }, "text/html");
}

AddDataFrameFormatter();

Carrega e extrai todos os dados de suas devidas colunas e montra um dataframe com as mesmas colunas e suas respectivas tuplas e exibe a quantidade tuplas carregadas.

In [1]:
var mlContext = new MLContext(seed: 0);

var trainingDataView = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: TRAIN_DATA_FILEPATH,
    hasHeader: true,
    separatorChar: ',',
    allowQuoting: true,
    allowSparse: false
);

var sentimentColumnValues = trainingDataView.GetColumn<string>(nameof(ModelInput.Sentiment));
var idColumnValues = trainingDataView.GetColumn<float>(nameof(ModelInput.ID));
var dateColumnValues = trainingDataView.GetColumn<string>(nameof(ModelInput.Date));
var queryColumnValues = trainingDataView.GetColumn<string>(nameof(ModelInput.Query));
var userColumnValues = trainingDataView.GetColumn<string>(nameof(ModelInput.User));
var textColumnValues = trainingDataView.GetColumn<string>(nameof(ModelInput.Text));

var sentimentColumn = new StringDataFrameColumn(nameof(ModelInput.Sentiment), sentimentColumnValues);
var idColumn = new PrimitiveDataFrameColumn<float>(nameof(ModelInput.ID), idColumnValues);
var dateColumn = new StringDataFrameColumn(nameof(ModelInput.Date), dateColumnValues);
var queryColumn = new StringDataFrameColumn(nameof(ModelInput.Query), queryColumnValues);
var userColumn = new StringDataFrameColumn(nameof(ModelInput.User), userColumnValues);
var textColumn = new StringDataFrameColumn(nameof(ModelInput.Text), textColumnValues);

var df = new DataFrame(sentimentColumn, idColumn, dateColumn, queryColumn, userColumn, textColumn);

var count = $"Rows: {idColumnValues.Count()}";

count

Rows: 1600000

Exibe o dataframe.

In [1]:
df

0,0,1.4678104E+09,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1.4678107E+09,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1.467811E+09,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1.4678112E+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1.4678112E+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
5,0,1.4678113E+09,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1.4678116E+09,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1.4678116E+09,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?"
8,0,1.4678118E+09,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1.467812E+09,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
10,0,1.4678124E+09,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


Utiliza o XPlot.Plotly para criar um gráfico pizza para agrupar as tuplas pelo valor do sentimento e, como dito anteriorte, aqui foi verificado a polarização binária dos dados.

In [1]:
var groupedSentimentColumnValues = sentimentColumnValues.GroupBy(_ => _);

var pieChart = new Graph.Pie {
    values = groupedSentimentColumnValues.Select(_ => _.Count()),
    labels = groupedSentimentColumnValues.Select(_ => _.Key),
};

var piePlot = Chart.Plot(pieChart);

piePlot

Cria um método para fornecer um *pipeline* de treinamento de dados que consiste em mapear os valores da coluna sentimento em forma de identificadores criando um *input* dos textos relacionados ao sentimento atribuído realizando ao final a normalização dos valores máximos e minímos para o valor de sentimento que, neste caso, ficou polarizado entre apenas os valores 0 e 4 que devido a isso, o [ML.NET Model Builder](https://docs.microsoft.com/en-us/dotnet/machine-learning/automate-training-with-model-builder) utilizou o método de treinamento de multiclasse, ou também conhecido como multi nomial, que permite fragmentar os dados e torna-los independentes, ou seja, ele não analisa a correlação entre eles e além disso utilizou a estratégia *one-versus-all* que dentro os modelos de classificação que, diferente do *one-versus-one* que faz a predição de uma classe a outra, a multi nomial pode prever para uma ou mais classes.

In [1]:
public IEstimator<ITransformer> BuildTrainingPipeline(MLContext mlContext){
    var dataProcessPipeline = mlContext.Transforms.Conversion.MapValueToKey("Sentiment", "Sentiment")
        .Append(mlContext.Transforms.Text.FeaturizeText("Text_tf", "Text"))
        .Append(mlContext.Transforms.CopyColumns("Features", "Text_tf"))
        .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
        .AppendCacheCheckpoint(mlContext);
        
    var trainer = mlContext.MulticlassClassification.Trainers.OneVersusAll(mlContext.BinaryClassification.Trainers.AveragedPerceptron(labelColumnName: "Sentiment", numberOfIterations: 10, featureColumnName: "Features"), labelColumnName: "Sentiment")
        .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel", "PredictedLabel"));

    var trainingPipeline = dataProcessPipeline.Append(trainer);

    return trainingPipeline;
}

Cria um método para a realização do treinamento do modelo.

In [1]:
public ITransformer TrainModel(MLContext mlContext, IDataView trainingDataView, IEstimator<ITransformer> trainingPipeline)
{
    Console.WriteLine("=============== Training  model ===============");

    ITransformer model = trainingPipeline.Fit(trainingDataView);

    Console.WriteLine("=============== End of training process ===============");
    return model;
}

Cria métodos para exibir métricas do modelo de classificação multi nomial.

In [1]:
public static void PrintMulticlassClassificationMetrics(MulticlassClassificationMetrics metrics)
{
    Console.WriteLine($"************************************************************");
    Console.WriteLine($"*    Metrics for multi-class classification model   ");
    Console.WriteLine($"*-----------------------------------------------------------");
    Console.WriteLine($"    MacroAccuracy = {metrics.MacroAccuracy:0.####}, a value between 0 and 1, the closer to 1, the better");
    Console.WriteLine($"    MicroAccuracy = {metrics.MicroAccuracy:0.####}, a value between 0 and 1, the closer to 1, the better");
    Console.WriteLine($"    LogLoss = {metrics.LogLoss:0.####}, the closer to 0, the better");
    for (int i = 0; i < metrics.PerClassLogLoss.Count; i++)
    {
        Console.WriteLine($"    LogLoss for class {i + 1} = {metrics.PerClassLogLoss[i]:0.####}, the closer to 0, the better");
    }
    Console.WriteLine($"************************************************************");
}

public void PrintMulticlassClassificationFoldsAverageMetrics(IEnumerable<TrainCatalogBase.CrossValidationResult<MulticlassClassificationMetrics>> crossValResults)
{
    var metricsInMultipleFolds = crossValResults.Select(r => r.Metrics);

    var microAccuracyValues = metricsInMultipleFolds.Select(m => m.MicroAccuracy);
    var microAccuracyAverage = microAccuracyValues.Average();
    var microAccuraciesStdDeviation = CalculateStandardDeviation(microAccuracyValues);
    var microAccuraciesConfidenceInterval95 = CalculateConfidenceInterval95(microAccuracyValues);

    var macroAccuracyValues = metricsInMultipleFolds.Select(m => m.MacroAccuracy);
    var macroAccuracyAverage = macroAccuracyValues.Average();
    var macroAccuraciesStdDeviation = CalculateStandardDeviation(macroAccuracyValues);
    var macroAccuraciesConfidenceInterval95 = CalculateConfidenceInterval95(macroAccuracyValues);

    var logLossValues = metricsInMultipleFolds.Select(m => m.LogLoss);
    var logLossAverage = logLossValues.Average();
    var logLossStdDeviation = CalculateStandardDeviation(logLossValues);
    var logLossConfidenceInterval95 = CalculateConfidenceInterval95(logLossValues);

    var logLossReductionValues = metricsInMultipleFolds.Select(m => m.LogLossReduction);
    var logLossReductionAverage = logLossReductionValues.Average();
    var logLossReductionStdDeviation = CalculateStandardDeviation(logLossReductionValues);
    var logLossReductionConfidenceInterval95 = CalculateConfidenceInterval95(logLossReductionValues);

    Console.WriteLine($"*************************************************************************************************************");
    Console.WriteLine($"*       Metrics for Multi-class Classification model      ");
    Console.WriteLine($"*------------------------------------------------------------------------------------------------------------");
    Console.WriteLine($"*       Average MicroAccuracy:    {microAccuracyAverage:0.###}  - Standard deviation: ({microAccuraciesStdDeviation:#.###})  - Confidence Interval 95%: ({microAccuraciesConfidenceInterval95:#.###})");
    Console.WriteLine($"*       Average MacroAccuracy:    {macroAccuracyAverage:0.###}  - Standard deviation: ({macroAccuraciesStdDeviation:#.###})  - Confidence Interval 95%: ({macroAccuraciesConfidenceInterval95:#.###})");
    Console.WriteLine($"*       Average LogLoss:          {logLossAverage:#.###}  - Standard deviation: ({logLossStdDeviation:#.###})  - Confidence Interval 95%: ({logLossConfidenceInterval95:#.###})");
    Console.WriteLine($"*       Average LogLossReduction: {logLossReductionAverage:#.###}  - Standard deviation: ({logLossReductionStdDeviation:#.###})  - Confidence Interval 95%: ({logLossReductionConfidenceInterval95:#.###})");
    Console.WriteLine($"*************************************************************************************************************");

}

public double CalculateStandardDeviation(IEnumerable<double> values)
{
    double average = values.Average();
    double sumOfSquaresOfDifferences = values.Select(val => (val - average) * (val - average)).Sum();
    double standardDeviation = Math.Sqrt(sumOfSquaresOfDifferences / (values.Count() - 1));
    return standardDeviation;
}

public double CalculateConfidenceInterval95(IEnumerable<double> values)
{
    double confidenceInterval95 = 1.96 * CalculateStandardDeviation(values) / Math.Sqrt((values.Count() - 1));
    return confidenceInterval95;
}

Cria método para avaliar o modelo de predição.

In [1]:
private void Evaluate(MLContext mlContext, IDataView trainingDataView, IEstimator<ITransformer> trainingPipeline){
    Console.WriteLine("=============== Cross-validating to get model's accuracy metrics ===============");
    var crossValidationResults = mlContext.MulticlassClassification.CrossValidate(trainingDataView, trainingPipeline, numberOfFolds: 5, labelColumnName: "Sentiment");
    PrintMulticlassClassificationFoldsAverageMetrics(crossValidationResults);
}

Cria método para salvar o treinamento do modelo para um arquivo.

In [1]:
private static void SaveModel(MLContext mlContext, ITransformer mlModel, string modelRelativePath, DataViewSchema modelInputSchema){
    Console.WriteLine($"=============== Saving the model  ===============");
    mlContext.Model.Save(mlModel, modelInputSchema, modelRelativePath);
    Console.WriteLine("The model is saved to {0}", modelRelativePath);
}

Cria método para a criação, treinamento, avaliação e salvação do modelo de treinamento.

In [1]:
public void CreateModel(MLContext mlContext, IDataView trainingDataView, string modelFilePath)
{
    IEstimator<ITransformer> trainingPipeline = BuildTrainingPipeline(mlContext);

    ITransformer mlModel = TrainModel(mlContext, trainingDataView, trainingPipeline);

    Evaluate(mlContext, trainingDataView, trainingPipeline);
    SaveModel(mlContext, mlModel, modelFilePath, trainingDataView.Schema);
}

O modelo só sera criado se não houver algum arquivo de "conhecimento" pré-salvo.

In [1]:
if (!File.Exists(MODEL_FILEPATH)){
    CreateModel(mlContext, trainingDataView, MODEL_FILEPATH);
}

=============== Training  model ===============


=============== End of training process ===============


=============== Cross-validating to get model's accuracy metrics ===============


*************************************************************************************************************


*       Metrics for Multi-class Classification model      


...

*       Average LogLoss:          .905  - Standard deviation: (.008)  - Confidence Interval 95%: (.008)


*       Average LogLossReduction: -.305  - Standard deviation: (.012)  - Confidence Interval 95%: (.012)


*************************************************************************************************************


=============== Saving the model  ===============


The model is saved to MLModel.zip


Cria um método para retornar um "motor" de predição baseado nos modelos definidos no ETL.

In [1]:
public PredictionEngine<ModelInput, ModelOutput> CreatePredictionEngine()
{
    var mlContext = new MLContext();

    var transformer = mlContext.Model.Load(MODEL_FILEPATH, out var modelInputSchema);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(transformer);

    return predictionEngine;
}

var predictionEngine = new Lazy<PredictionEngine<ModelInput, ModelOutput>>(CreatePredictionEngine);

Cria um método auxiliativo para a realização de predições.

In [1]:
public ModelOutput Predict(ModelInput input)
{
    ModelOutput result = predictionEngine.Value.Predict(input);
    return result;
}

Instancia um modelo de dado para ser utilizado durante a predição.

In [1]:
var sampleData = new ModelInput
{
    Text = "I love Machine Learning!"
};

Exibe resultados da predição.

In [1]:
var modelOutput = Predict(sampleData);

Console.WriteLine("Using model to make single prediction -- Comparing actual Sentiment with predicted Sentiment from sample data...\n\n");
Console.WriteLine($"Text: {sampleData.Text}");
Console.WriteLine($"\n\nPredicted Sentiment value {modelOutput.Prediction} \nPredicted Sentiment scores: [{String.Join(",", modelOutput.Score)}]\n\n");

Using model to make single prediction -- Comparing actual Sentiment with predicted Sentiment from sample data...




Text: I love Machine Learning!




Predicted Sentiment value 4 
Predicted Sentiment scores: [0.011912695,0.9880873]




## Conclusões

Realizando alguns testes com o modelo de predição, obtive resultados satisfatórios na maioria das vezes. O [ML.NET Model Builder](https://docs.microsoft.com/en-us/dotnet/machine-learning/automate-training-with-model-builder) tem como proposta realizar vários treinamentos utilizando diferentes algoritmos de predição e selecionar aquele que mais se destacou entre os demais. O método de classificação multi nomial nesse cenário teve os melhores resultados que, como explicado acima, permite um treinamento mais imparcial devido a ele não considerar a correlação entre os dados. Contudo, percebi que durantes os testes o modelo de predição quando não tem conhecimento de algum termo, ele retorna na maioria das vezes valores baixos e confessor que é até esperado esse comportamento, entretanto, como na proposta apresentada pela [Sentiment140](http://help.sentiment140.com/), quando foi realizado estudos analíticos com base nos dados, era esperados valores variando entre 0 para negativo, 2 para neutro e 4 para positivo, mas principalmente no gráfico pizza foi perceptível que houve uma polarização entre os dados em que, como consequência, fez o modelo de treinamento proposto utilizar uma estratégia próxima da classificação binário que, de certo modo, se não houvesse tal polarização talvez os resultados e até mesmo o modelo de treinamento poderia comportar melhor e realizar uma predição mais precisa dos dados.